In [ ]:
import pandas as pd
import ast

In [24]:
df = pd.read_csv('PD_experiments_results.csv')

In [4]:
number_of_games = {(0.5, 32) : [69, 71, 59], (0.5, 40) : [72, 71, 72], (0.5, 48) : [72, 68, 77],
                   (0.75, 32) : [29, 33, 27], (0.75, 40) : [34, 47, 23], (0.75, 48) : [35, 29, 32]}

In [ ]:
subjects = {(0.5, 32) : [16, 12, 16], (0.5, 40) : [14, 16, 20], (0.5, 48) : [14, 14, 18],
            (0.75, 32) : [14, 16, 14], (0.75, 40) : [12, 14, 12], (0.75, 48) : [16, 12, 16]}

In [5]:
# Function to get reward based on experiment and supergame
def get_reward(experiment):
    if experiment <= 5:
        return 32
    elif experiment >= 6 and experiment <= 11:
        return 40
    elif experiment >= 12 and experiment <= 17:
        return 48
    return None  # If no match found, return None

In [26]:
df['Reward'] = df.apply(lambda row: get_reward(row['Experiment']), axis=1)

In [27]:
# Convert the 'Data' column from string to dictionary using ast.literal_eval
df['Data'] = df['Data'].apply(ast.literal_eval)

# Extract the relevant columns (decisions, scores, etc.) into separate columns
df['Round'] = df['Data'].apply(lambda x: x['Round'])
df['Prisoner 1 Decision'] = df['Data'].apply(lambda x: x['Prisoner 1 LLM Response'])
df['Prisoner 2 Decision'] = df['Data'].apply(lambda x: x['Prisoner 2 LLM Response'])
df['Prisoner 1 Score'] = df['Data'].apply(lambda x: x['Prisoner 1 Score'])
df['Prisoner 2 Score'] = df['Data'].apply(lambda x: x['Prisoner 2 Score'])
df['Probability of Game Continuing'] = df['Data'].apply(lambda x: x['Probability of Game Continuing'])

In [35]:
df.head()

,Data,Experiment,Supergame,Subject Pair,Reward,Round,Prisoner 1 Decision,Prisoner 2 Decision,Prisoner 1 Score,Prisoner 2 Score,Probability of Game Continuing,Player 1 Cooperates,Player 2 Cooperates
0,"{'Round': 1, 'Prisoner 1 ID': 11, 'Prisoner 2 ...",0,1,1,32,1,Betray,Cooperate,50.0,12.0,0.5,0,1.0
1,"{'Round': 2, 'Prisoner 1 ID': 11, 'Prisoner 2 ...",0,1,1,32,2,Betray,Cooperate,100.0,24.0,0.5,0,1.0
2,"{'Round': 3, 'Prisoner 1 ID': 11, 'Prisoner 2 ...",0,1,1,32,3,Cooperate,Cooperate,132.0,56.0,0.5,1,1.0
3,"{'Round': 1, 'Prisoner 1 ID': 0, 'Prisoner 2 I...",0,1,2,32,1,Betray,Betray,25.0,25.0,0.5,0,0.0
4,"{'Round': 2, 'Prisoner 1 ID': 0, 'Prisoner 2 I...",0,1,2,32,2,Betray,Betray,50.0,50.0,0.5,0,0.0


In [33]:
df.shape

(2517, 13)

In [28]:
# Now let's define a function to classify decisions: 1 for Cooperation, 0 for Defection
def classify_decision(decision):
    if decision == 'Cooperate':
        return 1
    elif decision == 'Betray':
        return 0
    else:
        return None  # Handle other decision values if needed

# Apply the classification to the 'Prisoner 1 Decision' and 'Prisoner 2 Decision'
df['Player 1 Cooperates'] = df['Prisoner 1 Decision'].apply(classify_decision)
df['Player 2 Cooperates'] = df['Prisoner 2 Decision'].apply(classify_decision)

# Calculate cooperation rates per round
cooperation_rate_per_round = df.groupby('Round')[['Player 1 Cooperates', 'Player 2 Cooperates']].mean()


In [ ]:
# Define function to compute cooperation rate
def cooperation_rate(group):
    return group[['Player 1 Cooperates', 'Player 2 Cooperates']].mean().mean()

# Group by Reward and Probability of Game Continuing and compute average cooperation rate
cooperation_rate_by_reward_and_probability = df.groupby(['Reward', 'Probability of Game Continuing']).apply(cooperation_rate).reset_index(name='Cooperation Rate')

In [30]:
cooperation_rate_by_reward_and_probability.groupby(['Probability of Game Continuing'])['Cooperation Rate'].mean()

Probability of Game Continuing
0.50    0.603750
0.75    0.570826
Name: Cooperation Rate, dtype: float64